In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import scvelo as scv

In [2]:
adata = scv.datasets.pancreas()
adata

AnnData object with n_obs × n_vars = 3696 × 27998
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score'
    var: 'highly_variable_genes'
    uns: 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced'
    obsp: 'distances', 'connectivities'

In [3]:
# set(adata.obs['clusters'])

In [4]:
scv.pp.filter_genes(adata, min_shared_counts=20)
scv.pp.filter_genes_dispersion(adata, n_top_genes=2000)

Filtered out 20801 genes that are detected 20 counts (shared).
Extracted 2000 highly variable genes.


In [5]:
def volcano_data(adata, clusters, cell_name):
    s = adata.to_df(layer='spliced')
    u = adata.to_df(layer='unspliced')
    df = pd.DataFrame(adata.obs)
    cell_gene = df.index[df[clusters] == cell_name].tolist()
    dfs = s.loc[cell_gene].T
    dfu = u.loc[cell_gene].T
    mean_s = dfs.mean(axis=1)
    mean_u = dfu.mean(axis=1)
    gene = list(dfs.index)
    ge = pd.DataFrame(gene, columns=['Gene'])
    l2fc = np.array(np.log2(mean_s/mean_u))
    ge["l2fc"] = l2fc
    p_value = stats.ttest_ind(np.array(s.loc[cell_gene]),np.array(u.loc[cell_gene]))
    ge["pv"]= p_value[1]
    ge.fillna(0, inplace=True)
    temp = ge[(ge.l2fc!=0) & (ge.pv!=0)]
    exgene = np.array(temp.index)
    res_list = [np.array(ge['Gene'])[i] for i in exgene]    
    EXG = pd.DataFrame(res_list, columns=['exp_Gene'])
    EXG.to_csv('expgene.csv')
    print("Data save as name expgene.csv")
    ge.to_csv('volcano.csv')
    print("Data save as name volcano.csv") 

In [9]:
volcano_data(adata, clusters='clusters', cell_name='Beta')

Data save as name expgene.csv
Data save as name volcano.csv


In [10]:
df = pd.read_csv("volcano.csv")
df

,Unnamed: 0,Gene,l2fc,pv
0,0,Snhg6,2.834576,1.191276e-15
1,1,Lactb2,0.623437,4.829411e-02
2,2,Sbspon,1.584963,3.168992e-01
3,3,Pkhd1,-4.187627,9.206254e-24
4,4,Mcm3,inf,1.623448e-04
...,...,...,...,...
1995,1995,Tmem27,5.955650,0.000000e+00
1996,1996,Gpm6b,-2.000000,1.790842e-01
1997,1997,Ddx3y,-0.227410,2.398149e-01
1998,1998,Eif2s3y,0.377070,1.216073e-01


In [11]:
df2 = pd.read_csv("expgene.csv")
df2

,Unnamed: 0,exp_Gene
0,0,Snhg6
1,1,Lactb2
2,2,Sbspon
3,3,Pkhd1
4,4,Mcm3
...,...,...
1530,1530,Ap1s2
1531,1531,Gpm6b
1532,1532,Ddx3y
1533,1533,Eif2s3y
